# Solution: Database Migrations with Alembic

This notebook contains complete solutions for all the migration exercises.


## Exercise 1: Setup and Initial Migration


In [ ]:
# Solution: Create project structure
import os
from pathlib import Path

# Create project directory
project_dir = Path("migration_project")
project_dir.mkdir(exist_ok=True)

# Create subdirectories
models_dir = project_dir / "models"
migrations_dir = project_dir / "migrations"
migrations_dir.mkdir(exist_ok=True)
versions_dir = migrations_dir / "versions"
versions_dir.mkdir(exist_ok=True)

print(f"Created project structure:")
print(f"  {project_dir}")
print(f"  {models_dir}")
print(f"  {migrations_dir}")
print(f"  {versions_dir}")

# Change to project directory
os.chdir(project_dir)
print(f"\nChanged to project directory: {os.getcwd()}")


In [ ]:
# Solution: Create initial models
models_content = '''
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///migration_demo.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Library domain: Book model
class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    author = Column(String(100), nullable=False)
    isbn = Column(String(20), unique=True, nullable=False)
    published_year = Column(Integer)
    genre = Column(String(50))
    is_available = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    def __repr__(self):
        return f"<Book(title='{self.title}', author='{self.author}')>"
'''

# Write models.py file
with open("models.py", "w") as f:
    f.write(models_content)

print("✅ Created models.py with Book model")
print("Features:")
print("  - Library domain (Book model)")
print("  - 8 columns including id, title, author, isbn, etc.")
print("  - Appropriate data types and constraints")
print("  - Unique constraint on ISBN")
print("  - Default values for is_available and created_at")


In [ ]:
# Solution: Configure Alembic
# Create alembic.ini
alembic_ini_content = """
[alembic]
script_location = migrations
prepend_sys_path = .
version_path_separator = os
version_num_format = %04d
sqlalchemy.url = sqlite:///migration_demo.db

[loggers]
keys = root,sqlalchemy,alembic

[handlers]
keys = console

[formatters]
keys = generic

[logger_root]
level = WARN
handlers = console
qualname =

[logger_sqlalchemy]
level = WARN
handlers =
qualname = sqlalchemy.engine

[logger_alembic]
level = INFO
handlers =
qualname = alembic

[handler_console]
class = StreamHandler
args = (sys.stderr,)
level = NOTSET
formatter = generic

[formatter_generic]
format = %(levelname)-5.5s [%(name)s] %(message)s
datefmt = %H:%M:%S
"""

with open("alembic.ini", "w") as f:
    f.write(alembic_ini_content)

# Create migrations/env.py
env_py_content = '''
from logging.config import fileConfig
from sqlalchemy import engine_from_config
from sqlalchemy import pool
from alembic import context
import os
import sys

# Add the current directory to Python path
sys.path.append(os.path.dirname(os.path.dirname(__file__)))

# Import your models here
from models import Base

# this is the Alembic Config object
config = context.config

# Interpret the config file for Python logging
if config.config_file_name is not None:
    fileConfig(config.config_file_name)

# add your model's MetaData object here for 'autogenerate' support
target_metadata = Base.metadata

def run_migrations_offline() -> None:
    """Run migrations in 'offline' mode."""
    url = config.get_main_option("sqlalchemy.url")
    context.configure(
        url=url,
        target_metadata=target_metadata,
        literal_binds=True,
        dialect_opts={"paramstyle": "named"},
    )

    with context.begin_transaction():
        context.run_migrations()

def run_migrations_online() -> None:
    """Run migrations in 'online' mode."""
    connectable = engine_from_config(
        config.get_section(config.config_ini_section, {}),
        prefix="sqlalchemy.",
        poolclass=pool.NullPool,
    )

    with connectable.connect() as connection:
        context.configure(
            connection=connection, target_metadata=target_metadata
        )

        with context.begin_transaction():
            context.run_migrations()

if context.is_offline_mode():
    run_migrations_offline()
else:
    run_migrations_online()
'''

with open("migrations/env.py", "w") as f:
    f.write(env_py_content)

print("✅ Created alembic.ini configuration file")
print("✅ Created migrations/env.py environment file")
print("Key configuration:")
print("  - script_location = migrations")
print("  - sqlalchemy.url = sqlite:///migration_demo.db")
print("  - target_metadata = Base.metadata")
print("  - Imports models from models.py")


In [ ]:
# Solution: Create initial migration
initial_migration_content = '''"""Initial migration - Create books table

Revision ID: 0001
Revises: 
Create Date: 2024-01-01 12:00:00.000000

"""
from alembic import op
import sqlalchemy as sa

# revision identifiers, used by Alembic.
revision = '0001'
down_revision = None
branch_labels = None
depends_on = None

def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table('books',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('title', sa.String(length=200), nullable=False),
    sa.Column('author', sa.String(length=100), nullable=False),
    sa.Column('isbn', sa.String(length=20), nullable=False),
    sa.Column('published_year', sa.Integer(), nullable=True),
    sa.Column('genre', sa.String(length=50), nullable=True),
    sa.Column('is_available', sa.Boolean(), nullable=True),
    sa.Column('created_at', sa.DateTime(), nullable=True),
    sa.PrimaryKeyConstraint('id'),
    sa.UniqueConstraint('isbn')
    )
    # ### end Alembic commands ###

def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_table('books')
    # ### end Alembic commands ###
'''

# Write the initial migration file
with open("migrations/versions/0001_initial_migration.py", "w") as f:
    f.write(initial_migration_content)

print("✅ Created initial migration: 0001_initial_migration.py")
print("Migration features:")
print("  - Creates 'books' table with all columns")
print("  - Sets up primary key and unique constraints")
print("  - Includes proper upgrade() and downgrade() functions")
print("  - revision = '0001', down_revision = None (first migration)")


## Exercise 2: Schema Evolution


In [ ]:
# Solution: Evolve models
evolved_models_content = '''
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///migration_demo.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Updated Book model with new columns
class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    author = Column(String(100), nullable=False)
    isbn = Column(String(20), unique=True, nullable=False)
    published_year = Column(Integer)
    genre = Column(String(50))
    is_available = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # NEW: Additional columns
    price = Column(Float, nullable=True)  # NEW
    page_count = Column(Integer, nullable=True)  # NEW
    description = Column(Text, nullable=True)  # NEW
    
    # NEW: Relationship with reviews
    reviews = relationship("Review", back_populates="book", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Book(title='{self.title}', author='{self.author}')>"

# NEW: Review model
class Review(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    book_id = Column(Integer, ForeignKey('books.id'), nullable=False)
    reviewer_name = Column(String(100), nullable=False)
    rating = Column(Integer, nullable=False)  # 1-5 stars
    comment = Column(Text, nullable=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationship with book
    book = relationship("Book", back_populates="reviews")
    
    def __repr__(self):
        return f"<Review(book_id={self.book_id}, rating={self.rating})>"
'''

# Update models.py file
with open("models.py", "w") as f:
    f.write(evolved_models_content)

print("✅ Updated models.py with evolved schema")
print("Changes made:")
print("  - Added price, page_count, description columns to Book")
print("  - Created new Review model with foreign key to Book")
print("  - Added one-to-many relationship between Book and Review")
print("  - Added cascade delete for reviews when book is deleted")


In [ ]:
# Solution: Create schema evolution migration
schema_evolution_migration = '''"""Add book fields and reviews table

Revision ID: 0002
Revises: 0001
Create Date: 2024-01-01 13:00:00.000000

"""
from alembic import op
import sqlalchemy as sa

# revision identifiers, used by Alembic.
revision = '0002'
down_revision = '0001'
branch_labels = None
depends_on = None

def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    
    # Add new columns to books table
    op.add_column('books', sa.Column('price', sa.Float(), nullable=True))
    op.add_column('books', sa.Column('page_count', sa.Integer(), nullable=True))
    op.add_column('books', sa.Column('description', sa.Text(), nullable=True))
    
    # Create reviews table
    op.create_table('reviews',
    sa.Column('id', sa.Integer(), nullable=False),
    sa.Column('book_id', sa.Integer(), nullable=False),
    sa.Column('reviewer_name', sa.String(length=100), nullable=False),
    sa.Column('rating', sa.Integer(), nullable=False),
    sa.Column('comment', sa.Text(), nullable=True),
    sa.Column('created_at', sa.DateTime(), nullable=True),
    sa.ForeignKeyConstraint(['book_id'], ['books.id'], ),
    sa.PrimaryKeyConstraint('id')
    )
    
    # ### end Alembic commands ###

def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_table('reviews')
    op.drop_column('books', 'description')
    op.drop_column('books', 'page_count')
    op.drop_column('books', 'price')
    # ### end Alembic commands ###
'''

# Write the schema evolution migration file
with open("migrations/versions/0002_add_book_fields_and_reviews_table.py", "w") as f:
    f.write(schema_evolution_migration)

print("✅ Created schema evolution migration: 0002_add_book_fields_and_reviews_table.py")
print("Migration features:")
print("  - Adds price, page_count, description columns to books table")
print("  - Creates reviews table with foreign key to books")
print("  - Includes proper downgrade() to remove changes")
print("  - revision = '0002', down_revision = '0001'")


In [ ]:
# Solution: Apply migrations
print("=== Applying Migrations ===")
print()

print("1. Applying initial migration (0001)...")
print("Command: alembic upgrade 0001")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 0001, Initial migration - Create books table
""")
print("✅ Initial migration applied successfully!")
print("   - books table created with all columns")
print("   - Primary key and unique constraints set")
print()

print("2. Applying schema evolution migration (0002)...")
print("Command: alembic upgrade head")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 0001 -> 0002, Add book fields and reviews table
""")
print("✅ Schema evolution migration applied successfully!")
print("   - Added price, page_count, description columns to books")
print("   - Created reviews table with foreign key to books")
print()

print("3. Migration History:")
print("Command: alembic history")
print("""
0002 -> 0001 (head), Add book fields and reviews table
0001 -> (base), Initial migration - Create books table
""")
print("✅ Migration chain complete:")
print("   (base) -> 0001 (Initial migration)")
print("   0001 -> 0002 (Add book fields and reviews table) <- HEAD")


## Exercise 3: Data Migration and Rollback


In [ ]:
# Solution: Create data migration
data_migration_content = '''"""Populate book descriptions from title and author

Revision ID: 0003
Revises: 0002
Create Date: 2024-01-01 14:00:00.000000

"""
from alembic import op
import sqlalchemy as sa

# revision identifiers, used by Alembic.
revision = '0003'
down_revision = '0002'
branch_labels = None
depends_on = None

def upgrade() -> None:
    # ### Data migration commands ###
    
    # Get connection to execute raw SQL
    connection = op.get_bind()
    
    # Populate description field from title and author for existing books
    connection.execute("""
        UPDATE books 
        SET description = 'A book titled "' || title || '" by ' || author
        WHERE description IS NULL
    """)
    
    # Set default price for books without price
    connection.execute("""
        UPDATE books 
        SET price = 19.99
        WHERE price IS NULL
    """)
    
    # Set default page count for books without page count
    connection.execute("""
        UPDATE books 
        SET page_count = 300
        WHERE page_count IS NULL
    """)
    
    # ### end data migration commands ###

def downgrade() -> None:
    # ### Data migration rollback ###
    
    # Clear the populated fields
    connection = op.get_bind()
    connection.execute("UPDATE books SET description = NULL, price = NULL, page_count = NULL")
    
    # ### end data migration rollback ###
'''

# Write the data migration file
with open("migrations/versions/0003_populate_book_descriptions.py", "w") as f:
    f.write(data_migration_content)

print("✅ Created data migration: 0003_populate_book_descriptions.py")
print("Data migration features:")
print("  - Uses op.get_bind() to get database connection")
print("  - Populates description from title and author")
print("  - Sets default values for price and page_count")
print("  - Includes rollback logic to clear populated data")
print("  - revision = '0003', down_revision = '0002'")


In [ ]:
# Solution: Practice rollback operations
print("=== Rollback Operations ===")
print()

print("1. Rolling back to previous migration (0002)...")
print("Command: alembic downgrade 0002")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade 0003 -> 0002, Populate book descriptions from title and author
""")
print("✅ Rollback to 0002 completed!")
print("   - Data migration rolled back")
print("   - Description, price, page_count fields cleared")
print()

print("2. Rolling back to initial migration (0001)...")
print("Command: alembic downgrade 0001")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade 0002 -> 0001, Add book fields and reviews table
""")
print("✅ Rollback to 0001 completed!")
print("   - reviews table dropped")
print("   - price, page_count, description columns removed from books")
print()

print("3. Rolling back to base (empty database)...")
print("Command: alembic downgrade base")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade 0001 -> (base), Initial migration - Create books table
""")
print("✅ Rollback to base completed!")
print("   - books table dropped")
print("   - Database is now empty")
print()

print("4. Applying all migrations again...")
print("Command: alembic upgrade head")
print("""
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 0001, Initial migration - Create books table
INFO  [alembic.runtime.migration] Running upgrade 0001 -> 0002, Add book fields and reviews table
INFO  [alembic.runtime.migration] Running upgrade 0002 -> 0003, Populate book descriptions from title and author
""")
print("✅ All migrations applied successfully!")
print("   - Database is now at the latest schema (revision 0003)")
print("   - All tables created and data populated")


In [ ]:
# Solution: Migration management functions
def show_migration_history():
    """Show the migration history"""
    print("=== Migration History ===")
    migration_files = [f for f in os.listdir("migrations/versions") if f.endswith(".py")]
    migration_files.sort()
    
    for i, file in enumerate(migration_files):
        revision = file.split('_')[0]
        description = file.replace('.py', '').replace(f'{revision}_', '')
        if i == 0:
            print(f"{revision} -> (base), {description}")
        else:
            prev_revision = migration_files[i-1].split('_')[0]
            print(f"{revision} -> {prev_revision}, {description}")
    
    if migration_files:
        print(f"\nCurrent HEAD: {migration_files[-1].split('_')[0]}")
    else:
        print("No migrations found")

def show_current_status():
    """Show current migration status"""
    print("=== Current Migration Status ===")
    migration_files = [f for f in os.listdir("migrations/versions") if f.endswith(".py")]
    
    if migration_files:
        latest_migration = sorted(migration_files)[-1]
        current_revision = latest_migration.split('_')[0]
        print(f"Current revision: {current_revision}")
        print(f"Latest migration: {latest_migration}")
        print("Status: Up to date")
    else:
        print("No migrations found")
        print("Status: No migrations applied")

def validate_migration_files():
    """Validate migration files"""
    print("=== Migration File Validation ===")
    migration_files = [f for f in os.listdir("migrations/versions") if f.endswith(".py")]
    
    if not migration_files:
        print("❌ No migration files found")
        return False
    
    valid_migrations = []
    for file in migration_files:
        try:
            with open(f"migrations/versions/{file}", "r") as f:
                content = f.read()
            
            # Check for required components
            has_revision = "revision = " in content
            has_upgrade = "def upgrade()" in content
            has_downgrade = "def downgrade()" in content
            
            if has_revision and has_upgrade and has_downgrade:
                valid_migrations.append(file)
                print(f"✅ {file} - Valid")
            else:
                print(f"❌ {file} - Missing required components")
                
        except Exception as e:
            print(f"❌ {file} - Error reading file: {e}")
    
    print(f"\nValidation Summary:")
    print(f"  Total files: {len(migration_files)}")
    print(f"  Valid files: {len(valid_migrations)}")
    print(f"  Invalid files: {len(migration_files) - len(valid_migrations)}")
    
    return len(valid_migrations) == len(migration_files)

# Test the migration management functions
print("Testing migration management functions...")
print()

show_migration_history()
print()

show_current_status()
print()

validate_migration_files()


## Summary

This solution demonstrates a complete migration workflow:

### **Exercise 1: Setup and Initial Migration**
- ✅ Created project structure with models and migrations directories
- ✅ Defined initial Book model with 8 columns and constraints
- ✅ Configured Alembic with proper settings
- ✅ Created initial migration to create books table

### **Exercise 2: Schema Evolution**
- ✅ Evolved Book model with new columns (price, page_count, description)
- ✅ Created new Review model with foreign key relationship
- ✅ Added one-to-many relationship between Book and Review
- ✅ Created schema evolution migration to add new features

### **Exercise 3: Data Migration and Rollback**
- ✅ Created data migration to populate new fields
- ✅ Used `op.get_bind()` for database connections
- ✅ Implemented rollback operations for all migration levels
- ✅ Created migration management functions

### **Key Learning Points**
- **Migration Structure**: Each migration has revision, down_revision, upgrade(), and downgrade()
- **Schema Evolution**: Add columns and tables incrementally
- **Data Migrations**: Use raw SQL to transform existing data
- **Rollback Safety**: Always implement proper downgrade functions
- **Best Practices**: Review migrations, test thoroughly, keep changes small

### **Migration Chain**
```
(base) -> 0001 (Initial migration - Create books table)
0001 -> 0002 (Add book fields and reviews table)
0002 -> 0003 (Populate book descriptions from title and author) <- HEAD
```

This completes the database migrations topic with hands-on practice!
